In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
df = pd.read_csv('../data_clean/merged_total.csv')
df.fillna('unknown', inplace=True)
df.shape

(250666, 19)

### Embedding

In [3]:
import openai
import nltk
from dotenv import load_dotenv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\buyse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\buyse\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
load_dotenv("../.env")
openai.api_key = os.getenv('OPENAI_API_KEY')

embedding_model = "text-embedding-ada-002"

cat_cols = df.select_dtypes(include=['object']).columns
cat_cols = cat_cols[(cat_cols != 'visit_bounce') & (cat_cols != 'contact_contactpersoon_id') & (cat_cols != 'account_account_id')]

cat_cols

Index(['afspraak_keyphrases', 'account_keyphrases', 'campagne_keyphrases',
       'sessie_keyphrases', 'visit_keyphrases', 'mailing_keyphrases'],
      dtype='object')

In [5]:
def get_embedding(text):
    response = openai.Embedding.create(
    input=text,
    model=embedding_model
    )   
    return response['data'][0]['embedding']


def embed_col(df, col):
    unique_col = df[col].unique().tolist()
    dict_temp = {}

    for i in unique_col:
        dict_temp[i] = get_embedding(i)
    
    df[col+'_embed'] = df[col].map(dict_temp)
    df.drop(columns=[col], inplace=True)
    
    return df


def remove_stopwords(text):
    stop_words_nl = set(stopwords.words('dutch'))
    
    word_tokens = word_tokenize(text, language='dutch')

    result = [x for x in word_tokens if x not in stop_words_nl]

    seperator = ', '
    return seperator.join(result)


def team_name_change(text):
    teams_dict = {
        'jo': ' jong ondernemen ',
        'do': ' duurzaam ondernemen ',
        'in': ' innovatie digitalisering ',
        'io': ' internationaal ondernemen ',
        'ao': ' arbeidsmarkt ',
        'ex': ' expert ',
        'gr': ' groei ',
        'bb': ' belangenbehartiging ',
        'co': ' communicatie ',
        'nw': ' netwerking ',
        'ha': ' haven ',
        'ma': ' match '
    }
    word_tokens = word_tokenize(text, language='dutch')
    # apply dict to list
    result = [teams_dict.get(word, word) for word in word_tokens]
    # join list to string
    cleaned_list = ', '.join(result)
    # tokenize string
    tokenize_list = word_tokenize(cleaned_list, language='dutch')
    # remove comma
    tokenize_list_no_comma = [x for x in tokenize_list if x != ',']
    # join list to string and remove duplicates from list
    return ', '.join(list(set(tokenize_list_no_comma)))


def clean_text(df, cat_cols=cat_cols):

    df_copy = df.copy()

    for col in cat_cols:
        for row in range(len(df)):
            name_change = team_name_change(df[col][row])
            no_stopwords = remove_stopwords(name_change)
            tokenize_list = word_tokenize(no_stopwords, language='dutch')
            tokenize_list = [x for x in tokenize_list if x != ',']
            df_copy.at[row, col] = ', '.join(list(set(tokenize_list)))
    
    return df_copy


def flatten_vector_columns(df, col):
    # Flatten the 'vector_column' into a matrix (with padding)
    max_vector_length = max(len(vector) for vector in df[col])
    padded_matrix = np.array([vector + [0.0] * (max_vector_length - len(vector)) for vector in df[col]])
    return padded_matrix

duurt 11.5 minuten om df_clean te maken

In [6]:
df_clean = clean_text(df=df, cat_cols=cat_cols)

del df

In [16]:
for col in cat_cols:
    df_clean[col] = df_clean[col].str.replace(r'\d', '', regex=True).str.replace(', ,', ',')
    df_clean[col] = df_clean[col].apply(lambda x: 'unknown' if len(x) == 0 else x)

In [21]:
df_clean['campagne_keyphrases'][2]

'oost, netwerking, regio, offline, vlaanderen, netwerkevenement, nieuwjaarsreceptie'

Campagne_keyphrases embeddings: 6 minuten

In [ ]:
df_clean_emb1 = embed_col(df=df_clean, col='campagne_keyphrases')

In [24]:
len(df_clean_emb1['campagne_keyphrases_embed'][2])

1536

##### Met OpenAI embedding -> 1536 getallen per keyphrase

### CLUSTERING

In [6]:
from sklearn.cluster import DBSCAN

In [ ]:
# reduce the value from each embedded column to a single value
def reduce_embedding(embedded):
    return np.mean(embedded)

df_clean['campagne_naam_embedded'] = df_clean['campagne_naam_embedded'].apply(lambda x: reduce_embedding(x))
df_clean['visit_ip_embedded'] = df_clean['visit_ip_embedded'].apply(lambda x: reduce_embedding(x))
df_clean['afspraak_keyphrases_embedded'] = df_clean['afspraak_keyphrases_embedded'].apply(lambda x: reduce_embedding(x))
df_clean['mailing_name_embedded'] = df_clean['mailing_name_embedded'].apply(lambda x: reduce_embedding(x))
df_clean['mailing_onderwerp_embedded'] = df_clean['mailing_onderwerp_embedded'].apply(lambda x: reduce_embedding(x))

df_clean.shape

In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=5, ).fit(df_clean)

In [ ]:
labels_db = dbscan.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels_db)) - (1 if -1 in labels_db else 0)
n_noise_ = list(labels_db).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of clusters: 60
Estimated number of noise points: 9604


In [ ]:
dbscan.get_params(deep=True)

{'algorithm': 'auto',
 'eps': 0.5,
 'leaf_size': 30,
 'metric': 'euclidean',
 'metric_params': None,
 'min_samples': 5,
 'n_jobs': None,
 'p': None}